In [117]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [118]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

In [119]:
from sklearn.metrics import accuracy_score

In [120]:
train = pd.read_csv('Hotel-A-train.csv')

In [121]:
validate=pd.read_csv('Hotel-A-validation.csv')

In [122]:
train['Reservation_Status'].replace('Check-In', 1,inplace=True)
train['Reservation_Status'].replace('Canceled', 2,inplace=True)
train['Reservation_Status'].replace('No-Show', 3,inplace=True)

validate['Reservation_Status'].replace('Check-In', 1,inplace=True)
validate['Reservation_Status'].replace('Canceled', 2,inplace=True)
validate['Reservation_Status'].replace('No-Show', 3,inplace=True)

#### Dropping

In [123]:
train.drop(['Expected_checkin', 'Expected_checkout','Booking_date'], axis='columns', inplace=True)
validate.drop(['Expected_checkin', 'Expected_checkout','Booking_date'], axis='columns', inplace=True)

### Modeling Starts

In [124]:
x_t= train.drop('Reservation_Status',1)
y_t= train.Reservation_Status

In [125]:
x_v = validate.drop('Reservation_Status',1)
y_v= validate.Reservation_Status

In [126]:
x_t = pd.get_dummies(x_t)
x_v = pd.get_dummies(x_v)

In [127]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)


In [128]:
classifier.fit(x_t, y_t)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [130]:
pred_v = classifier.predict(x_v)

In [131]:
print(pred_v)

[1 1 1 ... 1 1 1]


In [132]:
type(pred_v)

numpy.ndarray

In [133]:
print(set(list(pred_v)))

{1, 2, 3}


In [134]:
accuracy_score(y_v,pred_v)

0.5863950527464532

### Testsing

In [135]:
test=pd.read_csv('Hotel-A-test.csv')
test['Income'].unique()

array(['25K --50K', '<25K', '>100K', '50K -- 100K'], dtype=object)

In [136]:
test.drop(['Expected_checkin', 'Expected_checkout','Booking_date'], axis='columns', inplace=True)

In [137]:
test = pd.get_dummies(test)

In [138]:
pred_test = classifier.predict(test)

In [139]:
submission=pd.DataFrame()
submission['Reservation-id']=test['Reservation-id']
submission['Reservation_Status']=pred_test

In [140]:
submission['Reservation_Status'].unique()

array([1, 2], dtype=int64)

In [141]:
submission.to_csv('Submission_l.csv',index=False)

In [142]:
from sklearn.metrics import classification_report,confusion_matrix

In [143]:
confusion_matrix(y_v, pred_v)

array([[1609,    0,    1],
       [ 738,    3,    0],
       [ 396,    2,    0]], dtype=int64)

In [144]:
print(classification_report(y_v, pred_v))

              precision    recall  f1-score   support

           1       0.59      1.00      0.74      1610
           2       0.60      0.00      0.01       741
           3       0.00      0.00      0.00       398

    accuracy                           0.59      2749
   macro avg       0.40      0.33      0.25      2749
weighted avg       0.51      0.59      0.44      2749

